In [1]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from collections import defaultdict
from datetime import datetime
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
user_test_path = '../beauty_data/processed_data/user_test.json'
user_valid_path = '../beauty_data/processed_data/user_valid.json'
user_train_path = '../beauty_data/processed_data/user_train.json'

In [3]:
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)
print(len(user_test_dict))

624


In [4]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
print(len(user_valid_dict))

624


In [5]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
print(len(user_train_dict))

624


In [6]:
print(user_valid_dict['0'])
print(user_test_dict['0'])
print(user_train_dict['0'])

[1, 2, 3, 4, 5, 5]
[2, 3, 4, 5, 5, 6]
[0, 1, 2, 3, 4, 5]


In [7]:
item_information_path = '../beauty_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict))

1216


In [8]:
ratings_path = '../beauty_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800


### Creating zeroshot data

In [9]:
def get_item_description(item_information):
    item_details = ""
    if 'title' in item_information:
        item_details += item_information['title']
    return item_details

In [10]:
zeroshot_dict = dict()
check_missed_items = []
for user, item_list in user_test_dict.items():  
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    prompt = "You are an expert beauty product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -\n"
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert beauty product recommender, predict whether the user will like or dislike the target product  - {target_item_details} in the format - Prediction: Yes or No"""
    zeroshot_dict[user] = prompt
    # break

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [11]:
zeroshot_test_path = './vanilla_zeroshot_dataset/vanilla_zeroshot_test.json'
with open(zeroshot_test_path, 'w+') as f:
    json.dump(zeroshot_dict, f)

In [12]:
zeroshot_dict = dict()
check_missed_items = []
for user, item_list in user_valid_dict.items():  
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    prompt = "You are an expert fashion product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -\n"
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert fashion product recommender, predict whether the user will like or dislike the target product  - {target_item_details} in the format - Prediction: Yes or No"""
    zeroshot_dict[user] = prompt

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [13]:
print(len(zeroshot_dict))

624


In [14]:
zeroshot_dict['34']

"You are an expert fashion product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -\nLiked Citre Shine Moisture Burst Shampoo - 16 fl oz\nLiked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz\nLiked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz\nLiked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz\nAs an expert fashion product recommender, predict whether the user will like or dislike the target product  - Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz in the format - Prediction: Yes or No"

In [15]:
total_len = 0
for user, prompt in zeroshot_dict.items():
    total_len += len(prompt.strip().split(' '))
print(total_len/len(zeroshot_dict))

136.3028846153846


In [16]:
users_very_long = []
for user, prompt in zeroshot_dict.items():
    if len(prompt.strip().split(' ')) > 250:
        users_very_long.append(user)

In [17]:
print(len(users_very_long), users_very_long)

1 ['59']


In [18]:
zeroshot_dict['59']

"You are an expert fashion product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -\nLiked So Beauty 10 Pieces 3D Rhinestone Dress Nail Art Slices Glitters DIY Decorations\nLiked Makartt 9 Sizes Q-French Pink &amp; White Tips Cutter Easy French Smiley Acrylic Gel Cutter Tool\nLiked WATER NAIL DECALS TRANSFERS STICKERS TATTOO ART SET #344 &amp; 172. **plus x48 nail tip guides!!** x24 NAUTICAL SHIP BOAT ANCHOR &amp; LILLY FLOWERS TATTOO WRAPS &amp; x48 FRENCH MANICURE TIP GUIDES! CAN BE USED WITH NATURAL GEL ACRYLIC STICK ON NAILS! OR WITH GLITTER DUST CAVIAR BEADS ALLOYS DECORATIONS CONFETTI FIMO SHAPES TAPE PENS RHINESTONES!\nLiked NAIL DECALS WATER TRANSFERS STICKERS TATTOO ART SET #605 &amp; 172. **plus x48 nail tip guides!!** x24 CHUNKY BLACK DJ CLUBBING HEADPHONES TATTOO WRAPS &amp; x48 FRENCH MANICURE TIP GUIDES! CAN BE USED WITH NAT

In [19]:
# print(zeroshot_dict['739'])

In [20]:
for key in users_very_long:
    zeroshot_dict.pop(key, None)
print(len(zeroshot_dict))

623


In [21]:
zeroshot_test_path = './vanilla_zeroshot_dataset/vanilla_zeroshot_valid.json'
with open(zeroshot_test_path, 'w+') as f:
    json.dump(zeroshot_dict, f)

In [22]:
ratings_df[ratings_df['user'] == int('0')]

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800
5,5,0,5.0,1404604800
6,5,0,5.0,1404604800
7,6,0,5.0,1461801600


In [23]:
test_prompt = zeroshot_dict['0']
print(test_prompt)
print("-"*100)
response = openai.completions.create(
    # model = "gpt-3.5-turbo-0125",
    model="gpt-3.5-turbo-instruct",
    prompt = test_prompt,
    temperature=0.3,
    max_tokens=16,
    # top_p=0.3,
    # frequency_penalty=0.5,
    # presence_penalty=0.5
)
print(response.choices[0].text)

You are an expert fashion product recommender. You are provided with a user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it. Please go through the list in order -
Liked Monoi - Monoi Pitate Jasmine 4 fl oz
Liked Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz
Liked Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz.
Liked Bonne Bell Smackers Bath and Body Starburst Collection
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
As an expert fashion product recommender, predict whether the user will like or dislike the target product  - Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz in the format - Prediction: Yes or No
----------------------------------------------------------------------------------------------------


Prediction: Yes
